# Modelo de classificação (S_GROUP_ID_1)

A seguir, apresentamos os testes realizados com os modelos de classificação durante esta Sprint. O objetivo desses modelos é prever, após a detecção de uma falha no carro, quais tipos de falhas são mais prováveis de ocorrer. Para isso, cada tipo de falha é abordado por um modelo binário. Nesta Sprint, concentramos nossos esforços na avaliação do desempenho do modelo para a classe "S_GROUP_ID_1", buscando identificar o modelo com a melhor performance. Com base nos resultados obtidos, planejamos aplicar os insights e aprimoramentos para os modelos das outras classes existentes.

Para conduzir os testes e comparar o desempenho dos diferentes modelos, utilizamos dois conjuntos de dados distintos. O primeiro conjunto de dados não incluiu as informações de torques fornecidas pelo parceiro, enquanto o segundo conjunto incorporou esses dados para treinar e aplicar o modelo. 

O processo de testes e as análises realizadas são detalhados a seguir, junto com as considerações e conclusões derivadas desses experimentos.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,GRU, Dropout, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam


## Teste sem dados de torque

A seguir, detalhamos os testes realizados utilizando redes neurais recorrentes LSTM e GRU nos dataframe sem dados de torque. O funcionamento dessas técnicas foi explicado em profundidade na documentação da Sprint 2, onde elas foram aplicadas ao modelo principal. Devido ao seu bom desempenho nessa etapa anterior, decidimos iniciar os testes desta Sprint utilizando essas RNNs como base.

In [5]:
# carregamento do dataframe
df = pd.read_csv("../../data/Merge_Falhas_Resultados.csv")
df["S_GROUP_ID_1"]

0        1
1        0
2        4
3        2
4        1
        ..
40149    4
40150    0
40151    4
40152    2
40153    2
Name: S_GROUP_ID_1, Length: 40154, dtype: int64

In [6]:
# Converter a coluna para binário
df['S_GROUP_ID_1'] = (df['S_GROUP_ID_1'] > 0).astype(int)
df['S_GROUP_ID_1']

0        1
1        0
2        1
3        1
4        1
        ..
40149    1
40150    0
40151    1
40152    1
40153    1
Name: S_GROUP_ID_1, Length: 40154, dtype: int32

In [7]:
# Separando as features (X) e o target (y)
X = df.drop(columns=['S_GROUP_ID_1', 'KNR'])  # 'KNR' é apenas um identificador, então deve ser removido
y = df['S_GROUP_ID_1']

In [8]:
# Separando em dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Converte X_train e X_test para arrays NumPy.
X_train = np.array(X_train)
X_test = np.array(X_test)

# Reestrutura X_train e X_test para ter 3 dimensões.
# A nova forma do array será (n_samples, n_features, 1)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [10]:
# Construção do modelo com LSTM
model_1 = Sequential()

model_1.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_1.add(LSTM(50, activation='relu'))
model_1.add(Dense(1))

model_1.compile(optimizer='adam', loss='mse')

c:\Users\Inteli\Documents\GitHub\2024-2A-T08-EC07-G01\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
# Treinamento do modelo
model_1.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 36s 32ms/step - loss: 300.8056 - val_loss: 2.7628
Epoch 2/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 28s 28ms/step - loss: 1.0809 - val_loss: 0.4644
Epoch 3/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 29s 29ms/step - loss: 0.4524 - val_loss: 0.4176
Epoch 4/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 30s 30ms/step - loss: 0.3700 - val_loss: 0.3696
Epoch 5/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 30s 30ms/step - loss: 0.3308 - val_loss: 0.5008
Epoch 6/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 30s 30ms/step - loss: 0.4190 - val_loss: 0.3706
Epoch 7/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 31s 31ms/step - loss: 0.3302 - val_loss: 0.3112
Epoch 8/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 41s 41ms/step - loss: 0.2777 - val_loss: 0.3475
Epoch 9/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 63s 63ms/step - loss: 0.3139 - val_loss: 0.2674
Epoch 10/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 40s 40ms/step - loss: 0.2692 - val_loss: 0.2564
Epoch 11/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 36s 36ms/step - loss: 2.9506 - val_loss: 0.

In [34]:
#Prever os dados de teste
y_pred_1 = model_1.predict(X_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_1 = (y_pred_1 > 0.5).astype(int)

# Calcular as principais métricas
accuracy_1 = accuracy_score(y_test, y_pred_classes_1)
precision_1= precision_score(y_test, y_pred_classes_1)
recall_1 = recall_score(y_test, y_pred_classes_1)
f1_1 = f1_score(y_test, y_pred_classes_1)

print(f"Accuracy: {accuracy_1:.4f}")
print(f"Precision: {precision_1:.4f}")
print(f"Recall: {recall_1:.4f}")
print(f"F1-Score: {f1_1:.4f}")


251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Accuracy: 0.6225
Precision: 0.6225
Recall: 1.0000
F1-Score: 0.7673


In [13]:
df["S_GROUP_ID_1"].describe()

count    40154.000000
mean         0.621507
std          0.485017
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: S_GROUP_ID_1, dtype: float64

In [14]:
# Construção do modelo com GRU
model_2 = Sequential()

model_2.add(GRU(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_2.add(GRU(50, activation='relu'))
model_2.add(Dense(1, activation='sigmoid'))

model_2.compile(optimizer='adam', loss='binary_crossentropy')

c:\Users\Inteli\Documents\GitHub\2024-2A-T08-EC07-G01\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
# Treinamento do modelo
model_2.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 103s 88ms/step - loss: 0.7814 - val_loss: 0.6714
Epoch 2/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 82s 82ms/step - loss: 0.6863 - val_loss: 0.7003
Epoch 3/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 81s 81ms/step - loss: 0.6787 - val_loss: 0.6698
Epoch 4/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 104s 104ms/step - loss: 0.6726 - val_loss: 0.6631
Epoch 5/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 101s 100ms/step - loss: 0.6654 - val_loss: 0.6612
Epoch 6/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 88s 88ms/step - loss: 0.6633 - val_loss: 0.6613
Epoch 7/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 86s 86ms/step - loss: 0.6740 - val_loss: 0.6691
Epoch 8/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 93s 92ms/step - loss: 0.6685 - val_loss: 0.6760
Epoch 9/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 99s 99ms/step - loss: 0.6664 - val_loss: 0.6605
Epoch 10/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 101s 100ms/step - loss: 0.6638 - val_loss: 0.6699
Epoch 11/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 114s 113ms/step - loss: 0.6612 - val_l

In [35]:
# Prever os dados de teste
y_pred_2 = model_2.predict(X_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_2 = (y_pred_2 > 0.5).astype(int)

# Calcular as principais métricas
accuracy_2 = accuracy_score(y_test, y_pred_classes_2)
precision_2 = precision_score(y_test, y_pred_classes_2)
recall_2 = recall_score(y_test, y_pred_classes_2)
f1_2 = f1_score(y_test, y_pred_classes_2)

print(f"Accuracy: {accuracy_2:.4f}")
print(f"Precision: {precision_2:.4f}")
print(f"Recall: {recall_2:.4f}")
print(f"F1-Score: {f1_2:.4f}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
Accuracy: 0.6225
Precision: 0.6225
Recall: 1.0000
F1-Score: 0.7673


Com base nos resultados obtidos, observamos que tanto os modelos LSTM quanto GRU apresentaram boa performance em termos de recall ao utilizar os dados de torque. No entanto, as demais métricas, como a acurácia, não tiveram o mesmo desempenho, o que nos leva a focar em melhorias nessas áreas. A próxima etapa envolve realizar testes com os dados sem torque e, em seguida, comparar os resultados dos modelos treinados com os dois conjuntos de dados. Isso nos permitirá decidir se devemos ou não manter os dados de torque e, a partir dessa decisão, testar novas técnicas para otimizar o modelo.

## Teste com dados de torque

A seguir, detalhamos os testes realizados utilizando redes neurais recorrentes LSTM e GRU nos dataframe com dados de torque. O funcionamento dessas técnicas foi explicado em profundidade na documentação da Sprint 2, onde elas foram aplicadas ao modelo principal. Devido ao seu bom desempenho nessa etapa anterior, decidimos iniciar os testes desta Sprint utilizando essas RNNs como base.

In [17]:
# carregando o dataframe
df2 = pd.read_csv("../../data/df_torques_falhas.csv")
df2

,KNR,%,CLICKS,DEG,GRAD,NM,V,\U00B0,KG,MIN,...,COR_6KA1,COR_6UA1,COR_A1A1,COR_K2A1,COR_K2K2,COR_O7O7,MOTOR_CWL,MOTOR_CWS,MOTOR_DHS,MOTOR_DRP
0,2023-2016173,0.616167,0.0,0.0,0.264989,0.074849,0.884665,0.470523,0.0,0.811406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-2026098,0.616167,0.0,0.0,0.462990,0.093920,0.919817,0.470523,0.0,0.811406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-2026162,0.616167,0.0,0.0,0.330866,0.094345,0.921463,0.470523,0.0,0.811406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-2026175,0.616167,0.0,0.0,0.436343,0.094345,0.894115,0.470523,0.0,0.811406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-2026215,0.616167,0.0,0.0,0.300888,0.094345,0.911870,0.470523,0.0,0.811406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49186,2024-2976009,0.616167,0.0,0.0,0.195855,0.430977,0.000000,0.470523,0.0,0.811406,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
49187,2024-2976010,0.616167,0.0,0.0,0.177955,0.429747,0.000000,0.470523,0.0,0.811406,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
49188,2024-2976011,0.616167,0.0,0.0,0.184092,0.424991,0.000000,0.470523,0.0,0.811406,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
49189,2024-2976012,0.616167,0.0,0.0,0.192237,0.436295,0.000000,0.470523,0.0,0.811406,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
# Converter a coluna para binário
df2['S_GROUP_ID_1'] = (df2['S_GROUP_ID_1'] > 0).astype(int)
df2['S_GROUP_ID_1']



0        0
1        0
2        0
3        0
4        0
        ..
49186    1
49187    0
49188    1
49189    1
49190    1
Name: S_GROUP_ID_1, Length: 49191, dtype: int32

In [19]:
# Excluindo as cplunas com dados NaN
df2 = df2.dropna()
df2.shape

(40152, 49)

In [20]:
# Separando as features (X) e o target (y)
X2 = df2.drop(columns=['S_GROUP_ID_1', 'KNR'])  # 'KNR' é apenas um identificador, então deve ser removido
y2 = df2['S_GROUP_ID_1']


In [22]:
# Separando em dados de treino e teste
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [25]:
# Converte X_train e X_test para arrays NumPy, caso ainda não sejam.
X2_train = np.array(X2_train)
X2_test = np.array(X2_test)
# Reestrutura X_train e X_test para ter 3 dimensões.
# A nova forma do array será (n_samples, n_features, 1)
X2_train = X2_train.reshape((X2_train.shape[0], X2_train.shape[1], 1))
X2_test = X2_test.reshape((X2_test.shape[0], X2_test.shape[1], 1))


In [26]:
# Construção do modelo com LSTM
model_1_2 = Sequential()

model_1_2.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(X2_train.shape[1], 1)))
model_1_2.add(LSTM(50, activation='relu'))
model_1_2.add(Dense(1))

model_1_2.compile(optimizer='adam', loss='mse')

c:\Users\Inteli\Documents\GitHub\2024-2A-T08-EC07-G01\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [28]:
# Treinamento do modelo
model_1_2.fit(X2_train, y2_train, epochs=100, batch_size=32, validation_data=(X2_test, y2_test))

Epoch 1/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 51s 51ms/step - loss: 113.5401 - val_loss: 10.2416
Epoch 2/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 35s 34ms/step - loss: 8.1171 - val_loss: 4.3603
Epoch 3/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 24s 24ms/step - loss: 20.1039 - val_loss: 1.4746
Epoch 4/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 24s 24ms/step - loss: 1.2708 - val_loss: 0.9603
Epoch 5/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 56s 55ms/step - loss: 0.9352 - val_loss: 0.9231
Epoch 6/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 73s 72ms/step - loss: 0.9554 - val_loss: 0.6711
Epoch 7/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 77s 77ms/step - loss: 0.7078 - val_loss: 0.5205
Epoch 8/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 57s 57ms/step - loss: 0.4704 - val_loss: 0.4181
Epoch 9/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 41s 40ms/step - loss: 0.3738 - val_loss: 0.3591
Epoch 10/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 42s 42ms/step - loss: 0.3463 - val_loss: 0.3918
Epoch 11/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 46s 45ms/step - loss: 0.3145 - val_loss: 

In [29]:
#Prever os dados de teste
y_pred_1_2 = model_1_2.predict(X2_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_1_2 = (y_pred_1_2 > 0.5).astype(int)

# Calcular as principais métricas
accuracy = accuracy_score(y2_test, y_pred_classes_1_2)
precision = precision_score(y2_test, y_pred_classes_1_2)
recall = recall_score(y2_test, y_pred_classes_1_2)
f1 = f1_score(y2_test, y_pred_classes_1_2)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step
Accuracy: 0.6047
Precision: 0.6207
Recall: 0.9388
F1-Score: 0.7473


In [30]:
# Construção do modelo com GRU
model_2_2 = Sequential()

model_2_2.add(GRU(50, activation='relu', return_sequences=True, input_shape=(X2_train.shape[1], 1)))
model_2_2.add(GRU(50, activation='relu'))
model_2_2.add(Dense(1, activation='sigmoid'))

model_2_2.compile(optimizer='adam', loss='binary_crossentropy')

c:\Users\Inteli\Documents\GitHub\2024-2A-T08-EC07-G01\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [36]:
# Treinamento do modelo
model_2_2.fit(X2_train, y2_train, epochs=100, batch_size=32, validation_data=(X2_test, y2_test))

Epoch 1/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 50s 49ms/step - loss: 0.6620 - val_loss: 0.6620
Epoch 2/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 50s 50ms/step - loss: 0.6615 - val_loss: 0.6614
Epoch 3/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 50s 50ms/step - loss: 0.6966 - val_loss: 0.6960
Epoch 4/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 52s 51ms/step - loss: 0.6715 - val_loss: 0.6679
Epoch 5/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 51s 51ms/step - loss: 0.6617 - val_loss: 0.6621
Epoch 6/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 50s 50ms/step - loss: 0.6635 - val_loss: 0.6603
Epoch 7/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 51s 51ms/step - loss: 0.6610 - val_loss: 0.6599
Epoch 8/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 50s 50ms/step - loss: 0.6629 - val_loss: 0.6622
Epoch 9/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 50s 50ms/step - loss: 0.6622 - val_loss: 0.6594
Epoch 10/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 51s 51ms/step - loss: 0.6585 - val_loss: 0.6593
Epoch 11/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 50s 50ms/step - loss: 0.6594 - val_loss: 0.65

In [37]:
# Prever os dados de teste
y_pred_2_2 = model_2_2.predict(X2_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_2_2 = (y_pred_2_2 > 0.5).astype(int)

# Calcular as principais métricas
accuracy = accuracy_score(y2_test, y_pred_classes_2_2)
precision = precision_score(y2_test, y_pred_classes_2_2)
recall = recall_score(y2_test, y_pred_classes_2_2)
f1 = f1_score(y2_test, y_pred_classes_2_2)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
Accuracy: 0.6226
Precision: 0.6227
Recall: 0.9994
F1-Score: 0.7673


Com base nos resultados obtidos, observamos que tanto os modelos LSTM quanto GRU tiveram um bom desempenho em termos de recall ao utilizar os dados de torque. No entanto, outras métricas, como a acurácia, não apresentaram o mesmo nível de performance, tanto com os dados de torque quanto com o dataframe sem esses dados. Ao comparar os dois dataframes, notamos que ambos tiveram um desempenho muito similar, o que indica que ambos podem ser utilizados nos modelos. No entanto, para os próximos testes, optamos por trabalhar com o dataframe que inclui os dados de torque, pois ele apresentou métricas mais consistentes e fornece um conjunto mais rico de features.

Para melhorar as métricas observadas, aplicaremos as seguintes técnicas:

- Diferentes testes de balanceamento 
- Teste de outros tipos de modelos.
- Revisão e aprimoramento do tratamento dos dados


## Últimos teste realizados nessa Sprint


Ainda essa Sprint, realizamos testes com o balanceamento por undersampling com clusterização, utilizando os centróides de cada cluster. Utilizamos essa tecnica a partir do código abaixo, utilizando X2_balanced e y2_balanced para treinar e aplicar os modelos GRU e LSTM. Entretanto, as métricas diminuiram drasticamente, indicando que devem ser realizadas melhorias ou na forma de balanceamento, ou no tramaento dos dados que estamos utilizando. Para averiguar, basta utilizar o código a seguir antes do treinamento e aplicação dos modelos na etapa de testes com dados de torque.

In [21]:
# Separando as classes majoritária e minoritária com base no target y2
X2_majority = X2[y2 == 0]
X2_minority = X2[y2 == 1]

# Definindo o número de clusters como o tamanho da classe minoritária
n_clusters = len(X2_minority)

# Aplicando o K-Means à classe majoritária
kmeans = KMeans(n_clusters=n_clusters, random_state=1)
kmeans.fit(X2_majority)

# Pegando os centróides dos clusters
X2_majority_centroids = kmeans.cluster_centers_

# Combinando os centróides com a classe minoritária
X2_balanced = np.vstack((X2_majority_centroids, X2_minority))
y2_balanced = np.hstack((np.zeros(len(X2_majority_centroids)), np.ones(len(X2_minority))))


Além disso, tentamos também aplicar a seguinte rede neural convolucional, para testar a performance do modelo. Infelizmente, não obtivemos resultados tão positivos quanto os dos modelo GRU e LSTM, porém, pretendemos focar em pesquisar e aplicar novos tipos de rede neural convolucional que possam se adequar melhor ao problema e trazer melhores resultados na próxima Sprint.

In [38]:
# Definir o modelo CNN
model_3 = Sequential()

# Primeira camada convolucional
model_3.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X2_train.shape[1], 1)))
model_3.add(MaxPooling1D(pool_size=2))
model_3.add(Dropout(0.3))

# Segunda camada convolucional
model_3.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model_3.add(MaxPooling1D(pool_size=2))
model_3.add(Dropout(0.3))

# Flatten para converter dados 2D em 1D
model_3.add(Flatten())

# Camada totalmente conectada
model_3.add(Dense(64, activation='relu'))
model_3.add(Dropout(0.5))

# Camada de saída para classificação binária
model_3.add(Dense(1, activation='sigmoid'))

# Compilação do modelo
model_3.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


c:\Users\Inteli\Documents\GitHub\2024-2A-T08-EC07-G01\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
# Treinamento do modelo
model_3.fit(X2_train, y2_train, epochs=100, batch_size=32, validation_data=(X2_test, y2_test))

Epoch 1/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5820 - loss: 2.2350 - val_accuracy: 0.6227 - val_loss: 0.6653
Epoch 2/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6247 - loss: 0.6663 - val_accuracy: 0.6227 - val_loss: 0.6625
Epoch 3/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6262 - loss: 0.6614 - val_accuracy: 0.6227 - val_loss: 0.6625
Epoch 4/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6204 - loss: 0.6638 - val_accuracy: 0.6227 - val_loss: 0.6622
Epoch 5/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6161 - loss: 0.6660 - val_accuracy: 0.6227 - val_loss: 0.6622
Epoch 6/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6248 - loss: 0.6619 - val_accuracy: 0.6227 - val_loss: 0.6623
Epoch 7/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6228 - loss: 0.6640 - val_accuracy: 0.6227 - val_loss: 0.6623
Epoch 8/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6212 - loss: 0

In [40]:
# Prever os dados de teste
y_pred_3 = model_3.predict(X2_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_3 = (y_pred_3 > 0.5).astype(int)

# Calcular as principais métricas
accuracy = accuracy_score(y2_test, y_pred_classes_3)
precision = precision_score(y2_test, y_pred_classes_3)
recall = recall_score(y2_test, y_pred_classes_3)
f1 = f1_score(y2_test, y_pred_classes_3)

# Exibir as métricas
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.6227
Precision: 0.6227
Recall: 1.0000
F1-Score: 0.7675
